# 2D Acoustic Wave
---
- Model Setup: [this link](https://github.com/devitocodes/devito/blob/master/examples/seismic/tutorials/01_modelling.ipynb)

- target: $c^2$.

- rescaling: $x' = x/1000, z' = z/1000, t' = t/1000$.

---

In [1]:
NAME = "0714_single_source_simplest_pretrain_source"

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("../../")
from Seismic_wave_inversion_PINN.tf_model_utils import *
from Seismic_wave_inversion_PINN.data_utils import *
from Seismic_wave_inversion_PINN.models import *

plt.rcParams.update(plt.rcParamsDefault)
tf.keras.backend.set_floatx("float32")

%load_ext autoreload
%autoreload 2

print(tf.config.list_physical_devices('GPU'))

# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
x0 = z0 = t0 = 1000

In [4]:
c = 30.0
w0 = 0.1 # since (x, z, t) in [0, 1000]**3
lambda_1 = 1e-4
direct_layers = [3, 128, 128, 128, 128, 1]
direct_model = siren_model(direct_layers, c, w0, lambda_1)

inverse_boundary = 500.0 / z0
inverse_values = [2.25, 2.25] # c^2, initialization
inverse_model = step_constant_model(inverse_boundary, inverse_values, [False, True])

In [5]:
domain = [0.0, 1000.0]
T_max = 1000.0
x_s = 500.0
z_s = 20.0

# wave source
f0 = 0.01

@tf.function
def q(x, z, t, x_s, z_s):
	return tf.where(tf.math.logical_and(tf.equal(x, x_s), tf.equal(z, z_s)),
                (1.0-2*np.pi**2*f0**2*(t - 1/f0)**2)*tf.exp(-np.pi**2*f0**2*(t-1/f0)**2),
                0.0)

In [6]:
loss_func = keras.losses.MeanAbsoluteError()
loss_func_r = keras.losses.MeanAbsoluteError()
loss_func_d = keras.losses.MeanAbsoluteError()

def get_residue_loss(tape, p, c, x, z, t, x_s, z_s, initial = False): 
	dp_dt = tape.gradient(p, t)
	dp_dx = tape.gradient(p, x)
	dp_dz = tape.gradient(p, z)
	dp_dtt = tape.gradient(dp_dt, t)
	dp_dxx = tape.gradient(dp_dx, x)
	dp_dzz = tape.gradient(dp_dz, z)
	eq = loss_func_r((1/t0**2)*dp_dtt - c*((1/x0**2)*dp_dxx + (1/z0**2)*dp_dzz), c*q(x*x0, z*z0, t*t0, x_s*x0, z_s*z0))
	if initial:
		neumann = loss_func_r(dp_dt, 0.0)
		return eq, neumann
	else:
		return eq
	
def step(batch):
	with tf.GradientTape(persistent = True) as tape:
		tape.watch(batch)
		[x_c, z_c, t_c, x_c_s, z_c_s,
		x_i, z_i, t_i, x_i_s, z_i_s, p_i,
		x_d, z_d, t_d, x_d_s, z_d_s, p_d] = batch
		
		out_c = direct_model(tf.concat([x_c, z_c, t_c], axis = 1))
		c_c = inverse_model(z_c)
		out_i = direct_model(tf.concat([x_i, z_i, t_i], axis = 1))
		c_i = inverse_model(z_i)
		out_d = direct_model(tf.concat([x_d, z_d, t_d], axis = 1))
		c_d = inverse_model(z_d)
		
		loss_c = get_residue_loss(tape, out_c, c_c, x_c, z_c, t_c, x_c_s, z_c_s)
		loss_ireq, loss_irn = get_residue_loss(tape, out_i, c_i, x_i, z_i, t_i, x_i_s, z_i_s, True)
		loss_dr = get_residue_loss(tape, out_d, c_d, x_d, z_d, t_d, x_d_s, z_d_s)
		loss_iv = loss_func(out_i, p_i)
		loss_dv = loss_func_d(out_d, p_d)
		loss = w_c*loss_c + w_ireq*loss_ireq + w_irn*loss_irn + w_dr*loss_dr + w_iv*loss_iv + w_dv*loss_dv
	grads = tape.gradient(loss, direct_model.trainable_variables + inverse_model.trainable_variables)
	optimizer.apply_gradients(zip(grads, direct_model.trainable_variables + inverse_model.trainable_variables))
	return loss, loss_c, loss_ireq, loss_irn, loss_dr, loss_iv, loss_dv

def train(iterations, start_iteration, datasets, batch_sizes, print_every = 100, save_every = 10000, save_path = None):
# 	shuffled = lambda datasets, batch_sizes: [tf.data.Dataset.from_tensor_slices(tuple(d)).shuffle(buffer_size = d[0].shape[0], \
#                                                         reshuffle_each_iteration = True).repeat().batch(batch_size) \
#                                         for (d, batch_size) in zip(datasets, batch_sizes)]
	
# 	data_s = shuffled(datasets, batch_sizes)
	data_s = [batch_generator(dataset, batch_size) for (dataset, batch_size) in zip(datasets, batch_sizes)]
	iteration = start_iteration
# 	session = tf.compat.v1.Session()
# 	with session.as_default():
	for dc, di, dd in zip(*data_s):
		iteration += 1
		di = list(di)
		di[2] = tf.constant(transform(np.random.random(di[2].shape), 0.0, T_max*(iteration-start_iteration)/iterations), dtype = tf.float32)
		loss = step(list(dc) + list(di) + list(dd))

		if iteration % print_every == 0:
			print(loss)
			print("{}, Epoch: {}, Loss: {:.4e}, c: {:.4e}, ireq: {:.4e}, irn: {:.4e}, dr: {:.4e}, iv: {:.4e}, dv:{:.4e}".format(get_time(), \
								iteration, *loss))
			tf.print(inverse_model.values)
		if iteration % save_every == 0:
			direct_model.save("models/{}/{}/checkpoint_{}/direct_model".format(NAME, save_path, iteration))
			inverse_model.save("models/{}/{}/checkpoint_{}/inverse_model".format(NAME, save_path, iteration))
		if iteration == start_iteration + iterations:
			break

In [7]:
import pickle
with open("dataset_single_source.pkl", "rb") as file:
	[x, t, p, _, _] = pickle.load(file)
    
txzs_d = tensor_grid([t, x, [z_s], [x_s], [z_s]])
t_d, x_d, z_d, x_d_s, z_d_s = txzs_d[:, 0:1], txzs_d[:, 1:2], txzs_d[:, 2:3], txzs_d[:, 3:4], txzs_d[:, 4:5]
p_d = p.reshape((-1, 1))

n_i = 2000
x_i, z_i = transform(np.random.random((n_i, 1)), *domain), transform(np.random.random((n_i, 1)), *domain)
t_i = np.zeros_like(x_i)
p_i = np.zeros_like(x_i)
x_i_s, z_i_s = np.tile([x_s], (n_i, 1)), np.tile([z_s], (n_i, 1))

n_cx = n_cz = n_ct = 0
x_c, z_c, t_c = np.linspace(*domain, n_cx).reshape((-1, 1)), np.linspace(*domain, n_cz).reshape((-1, 1)), np.linspace(0, T_max, n_ct).reshape((-1, 1))

n_source = 100000
x_source, z_source = np.tile([x_s], (n_source, 1)), np.tile([z_s], (n_source, 1))
t_source = transform(np.random.random((n_source, 1)), 0, 200)

x_c, z_c, t_c = np.vstack([x_c, x_source]), np.vstack([z_c, z_source]), np.vstack([t_c, t_source])
x_c_s, z_c_s = np.tile([x_s], x_c.shape), np.tile([z_s], z_c.shape)

datasets = ((x_c, z_c, t_c, x_c_s, z_c_s),
		(x_i, z_i, t_i, x_i_s, z_i_s, p_i),
		(x_d, z_d, t_d, x_d_s, z_d_s, p_d))

import pickle
with open("dataset_{}.pkl".format(NAME), "wb") as file:
    pickle.dump(datasets, file)

In [8]:
optimizer = keras.optimizers.Adam(learning_rate = 1e-5)
w_c, w_ireq, w_irn, w_dr = 1.0, 1.0, 1.0, 1.0
w_iv = 1.0
w_dv = 1.0

train(iterations = 1000000, start_iteration = 0, 
	  datasets = datasets, batch_sizes = (256, 128, 256), 
	  print_every = 2000, save_every = 50000, save_path = None)

(<tf.Tensor: shape=(), dtype=float32, numpy=2.3465178>, <tf.Tensor: shape=(), dtype=float32, numpy=1.9719213e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=1.8460221e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.890893>, <tf.Tensor: shape=(), dtype=float32, numpy=2.032439e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.50882685>, <tf.Tensor: shape=(), dtype=float32, numpy=0.94673944>)
2020/07/15, 16:05:08, Epoch: 2000, Loss: 2.3465e+00, c: 1.9719e-05, ireq: 1.8460e-05, irn: 8.9089e-01, dr: 2.0324e-05, iv: 5.0883e-01, dv:9.4674e-01
ListWrapper([2.25, 2.243186])
(<tf.Tensor: shape=(), dtype=float32, numpy=2.0426924>, <tf.Tensor: shape=(), dtype=float32, numpy=1.712655e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=1.5630498e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9092525>, <tf.Tensor: shape=(), dtype=float32, numpy=1.6114544e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.50316274>, <tf.Tensor: shape=(), dtype=float32, numpy=0.6302282>)
2020/07/15, 16:09:57, Epo

(<tf.Tensor: shape=(), dtype=float32, numpy=2.3393703>, <tf.Tensor: shape=(), dtype=float32, numpy=5.790154e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.984681e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.6749592e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=5.2131066e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=1.9003559e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=2.3393044>)
2020/07/15, 16:58:12, Epoch: 24000, Loss: 2.3394e+00, c: 5.7902e-10, ireq: 4.9847e-10, irn: 4.6750e-05, dr: 5.2131e-10, iv: 1.9004e-05, dv:2.3393e+00
ListWrapper([2.25, 2.17406583])
(<tf.Tensor: shape=(), dtype=float32, numpy=0.04783608>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0017997472>, <tf.Tensor: shape=(), dtype=float32, numpy=5.5206273e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=5.115712e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=5.017942e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=2.869482e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.04595648>)
2020/07/15

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/0714_single_source_simplest_pretrain_source/None/checkpoint_50000/direct_model/assets
INFO:tensorflow:Assets written to: models/0714_single_source_simplest_pretrain_source/None/checkpoint_50000/inverse_model/assets
(<tf.Tensor: shape=(), dtype=float32, numpy=0.41876793>, <tf.Tensor: shape=(), dtype=float32, numpy=5.357888e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=5.206879e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.7504866e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=4.8724114e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=3.427942e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.41868615>)
2020/07/15, 18:05:45, Epoch: 52000, Loss: 4.1877e-01, c: 5.3579e-10, ireq: 5.2069e-10, irn: 4.7505e-05, dr: 4.8724e-10, iv: 3.4279e-05, dv:4.1869e-01
ListWrapper([2.25, 2.08995557])
(<tf.Tensor: shape=(), dtype=float32, numpy=0.5152472>, <tf.Tensor: shape=(

(<tf.Tensor: shape=(), dtype=float32, numpy=0.06990626>, <tf.Tensor: shape=(), dtype=float32, numpy=4.9168947e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.324503e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.6514517e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=4.131594e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=2.4354373e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.06983539>)
2020/07/15, 19:08:33, Epoch: 78000, Loss: 6.9906e-02, c: 4.9169e-10, ireq: 4.3245e-10, irn: 4.6515e-05, dr: 4.1316e-10, iv: 2.4354e-05, dv:6.9835e-02
ListWrapper([2.25, 2.01180458])
(<tf.Tensor: shape=(), dtype=float32, numpy=0.5544772>, <tf.Tensor: shape=(), dtype=float32, numpy=5.135891e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=5.467943e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=5.4201675e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=5.303174e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=2.7646882e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.5543954>)
2020/07/1

(<tf.Tensor: shape=(), dtype=float32, numpy=0.50903785>, <tf.Tensor: shape=(), dtype=float32, numpy=4.1096765e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.0460976e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.6079054e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=4.056719e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=2.8626586e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.50896317>)
2020/07/15, 20:16:15, Epoch: 106000, Loss: 5.0904e-01, c: 4.1097e-10, ireq: 4.0461e-10, irn: 4.6079e-05, dr: 4.0567e-10, iv: 2.8627e-05, dv:5.0896e-01
ListWrapper([2.25, 1.92772925])
(<tf.Tensor: shape=(), dtype=float32, numpy=0.04517442>, <tf.Tensor: shape=(), dtype=float32, numpy=4.5153986e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.044729e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=4.6152953e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=4.744466e-10>, <tf.Tensor: shape=(), dtype=float32, numpy=2.9538667e-05>, <tf.Tensor: shape=(), dtype=float32, numpy=0.045098726>)
202

KeyboardInterrupt: 